In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")


def ber(a, b, fft_size=64):
    def qpsk_demod(a):
        a_real = np.real(a)
        a_imag = np.imag(a)

        a_real_bit = np.ones(a_real.shape)
        a_real_bit[a_real < 0] = 0

        a_imag_bit = np.ones(a_imag.shape)
        a_imag_bit[a_imag < 0] = 0

        a_bit = np.concatenate((a_imag_bit, a_real_bit), axis=0)
        a_bit = a_bit.reshape((1, -1), order='F')
        return a_bit
    
    carrier = np.hstack((np.arange(-26, -21), np.arange(-20, -7), np.arange(-6, 0),
                         np.arange(1, 7), np.arange(8, 21), np.arange(22, 27))) + fft_size // 2
    
    assert a.shape == b.shape
    a_list = a[:, carrier].reshape((1, -1))
    b_list = b[:, carrier].reshape((1, -1))

    a_bits = self.qpsk_demod(a_list)
    b_bits = self.qpsk_demod(b_list)

    error_count = np.sum(a_bits != b_bits)
    total_bits = a_bits.shape[1]
        
    ber = float(error_count) / float(total_bits)
    return ber

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [2]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [0]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23c
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 1
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std

In [3]:
#def ProcessLabel(label):
#    min_val, max_val = np.min(label), np.max(label)
#    label -= min_val
#    return label
#print(np.min(train_label))
#train_label = ProcessLabel(train_label)

In [4]:
nb_inputs  = 2
nb_hidden  = 96
nb_outputs = 2

time_step = 1e-3
nb_steps  = 100

batch_size = 32

import scipy.io

from dataset import Dataset, SpikingDataset, ARegDataset
train_data = ARegDataset(train_input, train_label, nb_inputs, nb_steps)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
print(len(train_data))
print(train_data[0][1].shape)
print(train_data[0][0].shape)

7521
(2,)
(8,)


In [5]:
class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 2)
        self.act = nn.ReLU()

    def forward(self,x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        x = self.act(x)
        x = self.fc4(x)
        return x

In [9]:
dnn_net = DNNNetwork(8, 2).to(device)

In [10]:
def train(trainloader, testloader, model, DNN_model, lr=2e-3, nb_epochs=10):
    #params = [w1,w2]
    optimizer = torch.optim.Adam(DNN_model.parameters(), lr=lr, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    loss_fn = torch.nn.MSELoss()
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    DNN_model.train()
    
    for e in range(nb_epochs):
        print(e)
        local_loss = []
        for x_local, y_local in trainloader:
            x_local = x_local.float().to(device)
            y_local = y_local.float().to(device)
    
            #output = model(x_local)
            #output = output.flatten(start_dim=1)
            optimizer.zero_grad()
            output = DNN_model(x_local)
            loss_val = 100 * loss_fn(output, y_local)
            loss_val.backward()
            #print("AAAA: ", DNN_model.fc2.weight)
            #print("BBBB: ", DNN_model.fc1.weight.grad)
            optimizer.step()
            local_loss.append(loss_val.item())
            
        
        #if e % 1 == 30 and e != 0:
        #    print("Training accuracy: %.3f"%(compute_ber(trainloader, net, "train")))
        #    print("Test accuracy: %.3f"%(compute_ber(testloader, net, name)))
        scheduler.step()
        mean_loss = np.mean(local_loss)
        print(mean_loss)
        #print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        loss_hist.append(mean_loss)
        

In [12]:
train(train_loader, train_loader, dnn_net, dnn_net, lr=1e-2, nb_epochs=300)

0
2.373663954815622
1
2.1034454490547465
2
2.1808609483858286
3
2.1080262950795183
4
1.9465129713890916
5
1.779584568688425
6
1.8247970558892368
7
1.685951786778741
8
1.626614322573128
9
1.5619444118465406
10
1.5440970330925312
11
1.6083501372959907
12
1.6381293043999348
13
1.6143571149747251
14
1.5978078427961317
15
1.5677477534292108
16
1.5910550993630441
17
1.6298217919923492
18
1.5354205636402307
19
1.526179462544999
20
1.5661565636174153
21
1.5646725034309645
22
1.5063735386830266
23
1.4417925172690618
24
1.414823699174291
25
1.444418061082646
26
1.431042138171398
27
1.3755614677980794
28
1.553071953229985
29
1.878374686180535
30
1.6083596034322756
31
1.4910278062067799
32
1.4243283439743317
33
1.3698133050890293
34
1.3122761604391922
35
1.4115007235589674
36
1.3392519502569054
37
1.4835870436913634
38
1.4470641441264396
39
1.3192526546575256
40
1.3300257726114686
41
1.316703704580412
42
1.37579431031215
43
1.3452067187024375
44
1.438544025423668
45
1.306538727829012
46
1.29427535

In [13]:
dnn_net.eval()
error = 0.0
size = 20
for i in range(size):
    input, target = train_data[i]
    target = torch.tensor(target).float()
    input = torch.tensor(input).float()

    output = dnn_net(input.unsqueeze(dim=0).to(device))

    error += torch.sqrt((output - target) ** 2)
print(error / size)

tensor([[0.1089, 0.1195]], grad_fn=<DivBackward0>)
